<a href="https://colab.research.google.com/github/goldjunge3010/masterpraktikum/blob/main/Working_Example_Adding_Custom_Plot_To_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.7 MB/s eta 0:00:00


In [2]:
# import necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import re

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger
import wandb.apis.reports as wr

wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


In [3]:
# function to count sequence length
def count_seq_length(df: pd.DataFrame, seq_col: str) -> pd.Series:
    pattern = re.compile(r"\[UNIMOD:.*\]", re.IGNORECASE)
    df[seq_col].replace(pattern, "", inplace= True)
    return df[seq_col].str.len()

In [4]:
# count sequence lengths
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/goldjunge3010/masterpraktikum/main/third_pool_tresh_1_0_train.csv'
train_set = pd.read_csv(TRAIN_DATAPATH)
counts = count_seq_length(pd.read_csv(TRAIN_DATAPATH), "modified_sequence")

In [5]:
# Initialize WANDB
PROJECT = 'Custom_Plot_Working_Example'
RUN = "run_1"
wandb.init(project = PROJECT, name = RUN)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# log table to wandb
c = counts.to_frame()
table = wandb.Table(dataframe = c)

# create custom plot and log it
field = list(train_set.columns.values)[0]

hist = wandb.plot_table(
  vega_spec_name="master_praktikum/hist_pep_len",
  data_table = table,
  fields = {"value" : field}
)
wandb.log({"Histogram": hist})

# finish run
wandb.finish()

In [12]:
# create report
report = wr.Report(
    project = PROJECT,
    title = "Working custom plots in report!",
    description = "The result of pain, blood and tears."
)
report.save()

Report(project='Custom_Plot_Working_Example', entity='master_praktikum', title='Working custom plots in report!', description='The result of pain, blood and tears.', width='readable')

In [13]:
panels_data=[
  wr.CustomChart(
    query = {'summaryTable': {"tableKey" : "Histogram_table"}},
    chart_name='master_praktikum/hist_pep_len',
    chart_fields={'value': field}
  ),
]

In [14]:
ENTITY = wandb.apis.PublicApi().default_entity

data_hists = wr.PanelGrid(
    runsets=[
        wr.Runset(ENTITY, PROJECT),
    ],
    panels=
      panels_data
)

report.blocks = report.blocks[:1] +[wr.H1("Working custom plot"), data_hists] + [wr.Video(url="https://www.youtube.com/embed/dQw4w9WgXcQ")] + report.blocks[1:]

report.save()

Report(project='Custom_Plot_Working_Example', entity='master_praktikum', title='Working custom plots in report!', description='The result of pain, blood and tears.', width='readable', blocks=[H1(text=['Working custom plot']), PanelGrid(runsets=[Runset(entity='master_praktikum', project='Custom_Plot_Working_Example', name='Run set', query='', filters={'$or': [{'$and': []}]}, order=['-CreatedTimestamp'])], panels=[CustomChart(query={'runSets': {'runSets': '${runSets}', 'limit': 500}}, chart_name='master_praktikum/hist_pep_len', chart_fields={'value': 'modified_sequence'})]), Video(url='https://www.youtube.com/embed/dQw4w9WgXcQ')])